<a href="https://colab.research.google.com/github/Arup3201/Image-Classification-Tensorflow/blob/main/image-classification-tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

Download Dataset

In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d tongpython/cat-and-dog

 93% 202M/218M [00:00<00:00, 241MB/s]
100% 218M/218M [00:00<00:00, 246MB/s]


In [5]:
!mkdir ./cats-dogs
!unzip -q cat-and-dog.zip -d ./cats-dogs
!rm cat-and-dog.zip